## PROJECT INIT

- Handle imports and settings
- Initialize DB

In [2]:
import pathlib

import holidays
import numpy as np
import pandas as pd
import sklearn
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import (
    OneHotEncoder,
    OrdinalEncoder,
    StandardScaler,
    PolynomialFeatures,
)
from sqlalchemy import create_engine

from utils.config import load_config
from utils.data import initialize_project

config_path = pathlib.Path.cwd().parent / "config.yaml"
config = load_config(config_path)
sklearn.set_config(transform_output="pandas")

db_uri = initialize_project(config=config, init_db=True)
engine = create_engine(db_uri)

Config loaded from /Users/haukesteffen/dev/TabularShenanigans/config.yaml.
Table 'competitions' already exists in the database.
Competition 'playground-series-s5e8' already exists in the database.
Table 'datasets' already exists in the database.
Dataset 'playground-series-s5e8-train' already exists in the 'datasets' table.
Dataset 'playground-series-s5e8-test' already exists in the 'datasets' table.


## FEATURE ENGINEERING

In [7]:
with engine.begin() as connection:
    train = pd.read_sql(
        sql=f"SELECT * FROM [{config.competition_name}-train]", con=connection
    ).convert_dtypes()
    test = pd.read_sql(
        sql=f"SELECT * FROM [{config.competition_name}-test]", con=connection
    ).convert_dtypes()

train_index = train[config.id_column]
test_index = test[config.id_column]
train = train.drop(columns=[config.id_column])
test = test.drop(columns=[config.id_column])

binary_features = ["default", "housing", "loan"]
categorical_features = ["job", "marital", "education", "contact", "poutcome"]
numerical_features = [
    "age",
    "balance",
    "duration",
    "campaign",
    "pdays",
    "previous",
]

# map months from string to integer
month_map = {
    "jan": 1,
    "feb": 2,
    "mar": 3,
    "apr": 4,
    "may": 5,
    "jun": 6,
    "jul": 7,
    "aug": 8,
    "sep": 9,
    "oct": 10,
    "nov": 11,
    "dec": 12,
}
train["month"] = train["month"].map(month_map)
test["month"] = test["month"].map(month_map)


# fix invalid day/month combinations
def fix_days_and_months(row):
    if row["day"] == 31 and row["month"] in [4, 6, 9, 11]:
        row["day"] = 30
    if row["day"] >= 29 and row["month"] == 2:
        row["day"] = 28
    return row


train = train.apply(fix_days_and_months, axis=1)
test = test.apply(fix_days_and_months, axis=1)


# convert day and month to datetime and add date-related features
train["date"] = pd.to_datetime(
    dict(year=2023, month=train["month"], day=train["day"]), errors="coerce"
)
test["date"] = pd.to_datetime(
    dict(year=2023, month=test["month"], day=test["day"]), errors="coerce"
)
train["day_of_year"] = train["date"].dt.dayofyear
test["day_of_year"] = test["date"].dt.dayofyear
train["day_of_week"] = train["date"].dt.dayofweek
test["day_of_week"] = test["date"].dt.dayofweek
train["week_of_year"] = train["date"].dt.isocalendar().week
test["week_of_year"] = test["date"].dt.isocalendar().week
train["quarter"] = train["date"].dt.quarter
test["quarter"] = test["date"].dt.quarter


# extract season from month
def get_season(month):
    if month in [12, 1, 2]:
        return "Winter"
    elif month in [3, 4, 5]:
        return "Spring"
    elif month in [6, 7, 8]:
        return "Summer"
    else:
        return "Autumn"


train["season"] = train["month"].apply(get_season)
test["season"] = test["month"].apply(get_season)
categorical_features += ["season"]


# extract holiday from date
portugal_holidays = holidays.Portugal(years=2023)
train["is_holiday"] = train["date"].isin(portugal_holidays).astype(int)
test["is_holiday"] = test["date"].isin(portugal_holidays).astype(int)
binary_features += ["is_holiday"]


# create cyclical features for date-related features
def create_cyclical_features(train, test, col):
    train[col + "_sin"] = np.sin(2 * np.pi * train[col] / train[col].max())
    train[col + "_cos"] = np.cos(2 * np.pi * train[col] / train[col].max())
    test[col + "_sin"] = np.sin(2 * np.pi * test[col] / train[col].max())
    test[col + "_cos"] = np.cos(2 * np.pi * test[col] / train[col].max())
    return train, test


train, test = create_cyclical_features(train, test, "day_of_week")
train, test = create_cyclical_features(train, test, "day")
train, test = create_cyclical_features(train, test, "day_of_year")
train, test = create_cyclical_features(train, test, "week_of_year")
train, test = create_cyclical_features(train, test, "month")
train, test = create_cyclical_features(train, test, "quarter")
numerical_features += [
    "day_of_week_sin",
    "day_of_week_cos",
    "day_sin",
    "day_cos",
    "day_of_year_sin",
    "day_of_year_cos",
    "week_of_year_sin",
    "week_of_year_cos",
    "month_sin",
    "month_cos",
    "quarter_sin",
    "quarter_cos",
]


# create flag if the client has been contacted before
# and map pdays to a more meaningful value
train["never_contacted_before"] = train.apply(
    lambda row: 1 if row["pdays"] == -1 else 0, axis=1
)
test["never_contacted_before"] = test.apply(
    lambda row: 1 if row["pdays"] == -1 else 0, axis=1
)
binary_features += ["never_contacted_before"]
max_pdays = train["pdays"].max()
train["pdays"] = train["pdays"].apply(lambda x: x if x != -1 else max_pdays)
test["pdays"] = test["pdays"].apply(lambda x: x if x != -1 else max_pdays)


# create features based on interactions
train["balance_per_age"] = train["balance"] / (train["age"])
test["balance_per_age"] = test["balance"] / (test["age"])
numerical_features += ["balance_per_age"]
train["housing_and_loan"] = train.apply(
    lambda row: "yes" if row["housing"] == "yes" and row["loan"] == "yes" else "no",
    axis=1,
)
test["housing_and_loan"] = test.apply(
    lambda row: "yes" if row["housing"] == "yes" and row["loan"] == "yes" else "no",
    axis=1,
)
binary_features += ["housing_and_loan"]

# drop original date-related features
train = train.drop(
    columns=[
        "day",
        "month",
        "date",
        "day_of_year",
        "day_of_week",
        "week_of_year",
        "quarter",
    ],
)
test = test.drop(
    columns=[
        "day",
        "month",
        "date",
        "day_of_year",
        "day_of_week",
        "week_of_year",
        "quarter",
    ],
)

# add polynomials of numerical features
poly = PolynomialFeatures(degree=(2, 2), interaction_only=True)
poly_train = poly.fit_transform(train[numerical_features]).drop(columns=["1"])
poly_test = poly.transform(test[numerical_features]).drop(columns=["1"])
train = pd.concat([train, poly_train], axis=1).copy(deep=True)
test = pd.concat([test, poly_test], axis=1).copy(deep=True)
numerical_features += poly_train.columns.to_list()

# feature consistency checks
features = train.drop(columns=[config.target_column]).columns.tolist()

# assert every feature is in one of the feature categories
assert set(features) == set(binary_features + categorical_features + numerical_features)

# assert every feature is in only one feature category
assert len(set(numerical_features) & set(binary_features)) == 0
assert len(set(binary_features) & set(categorical_features)) == 0
assert len(set(categorical_features) & set(numerical_features)) == 0

/var/folders/r7/1qf9bs6920bgntn0wmtsp71m0000gn/T/ipykernel_6540/1480945169.py:93: FutureWarning: The behavior of 'isin' with dtype=datetime64[ns] and castable values (e.g. strings) is deprecated. In a future version, these will not be considered matching by isin. Explicitly cast to the appropriate dtype before calling isin instead.
  train["is_holiday"] = train["date"].isin(portugal_holidays).astype(int)
/var/folders/r7/1qf9bs6920bgntn0wmtsp71m0000gn/T/ipykernel_6540/1480945169.py:94: FutureWarning: The behavior of 'isin' with dtype=datetime64[ns] and castable values (e.g. strings) is deprecated. In a future version, these will not be considered matching by isin. Explicitly cast to the appropriate dtype before calling isin instead.
  test["is_holiday"] = test["date"].isin(portugal_holidays).astype(int)


In [10]:
# split datasets into features/target, as well as train/validation
X_train_raw = train.drop(columns=[config.target_column])
y_train_raw = train[config.target_column]
X_test_raw = test.copy()

pipeline = Pipeline(
    [
        (
            "column_transform",
            ColumnTransformer(
                [
                    (
                        "binary_encode",
                        OrdinalEncoder(
                            handle_unknown="use_encoded_value", unknown_value=np.nan
                        ),
                        binary_features,
                    ),
                    (
                        "categorical_encode",
                        OneHotEncoder(
                            handle_unknown="ignore",
                            sparse_output=False,
                            drop="first",
                        ),
                        categorical_features,
                    ),
                    (
                        "scale_numerical",
                        StandardScaler(),
                        numerical_features,
                    ),
                ],
                remainder="drop",
                verbose_feature_names_out=False,
            ),
        ),
    ]
)

X_train = pipeline.fit_transform(X_train_raw)
X_test = pipeline.transform(X_test_raw)

In [11]:
train_preprocessed = pd.concat([train_index, X_train, y_train_raw], axis=1)
test_preprocessed = pd.concat([test_index, X_test], axis=1)

with engine.begin() as connection:
    train_preprocessed.to_sql(
        config.competition_name + "-train-preprocessed",
        connection,
        if_exists="replace",
        index=False,
    )
    test_preprocessed.to_sql(
        config.competition_name + "-test-preprocessed",
        connection,
        if_exists="replace",
        index=False,
    )